# Проект 3. Решить задачу DaNetQA / BoolQ

Можно решить как задачу для русского, так и для английского.

Либо провести эксперименты с многоязычной моделью

https://russiansuperglue.com/ru/tasks/task_info/DaNetQA

## Описание
Причинно-следственная связь, логический вывод, Natural Language Inference

DaNetQA - это набор да/нет вопросов с ответами и фрагментом текста, содержащим ответ. Все вопросы были написаны авторами без каких-либо искусственных ограничений.

Каждый пример представляет собой триплет (вопрос, фрагмент текста, ответ) с заголовком страницы в качестве необязательного дополнительного контекста.

Настройка классификации текстовых пар аналогична существующим задачам логического вывода (NLI)

### Тип задачи
Логика, Commonsense, Знания о мире. Бинарная классификация: true/false

# Imports

In [6]:
import pandas as pd
import numpy as np

# Подготовка данных

### Загрузка данных

In [7]:
def loadJSONL(path, name):
    df = pd.read_json(path, lines=True)
    df.drop('idx', axis=1, inplace=True)
    print(name)
    display(df.head())
    if (df.columns.values == 'label').any():
        s = np.unique(df['label'].to_numpy(), return_counts=True)[1]
        print(f"True answer: {s[1]}")
        print(f"False answer: {s[0]}")
        print("")
    return df

In [123]:
df_train = loadJSONL("DaNetQA/train.jsonl", "Train set")
df_validation = loadJSONL("DaNetQA/val.jsonl", "Validation set")
df_test = loadJSONL("DaNetQA/test.jsonl", "Test set:")


Train set


,question,passage,label
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False


True answer: 1061
False answer: 688

Validation set


,question,passage,label
0,Есть ли вода на марсе?,Гидросфера Марса — это совокупность водных зап...,True
1,Состоит ли англия в евросоюзе?,В полночь с 31 января на 1 февраля 2020 года п...,False
2,Действительно ли в ссср не было адвокатов?,Семён Львович Ария — советский и российский ю...,False
3,Была ли чума в оране?,"Чума — это и абсурд, что осмысливается как фор...",True
4,Был ли кетчуп в читосе?,Текущий каталог продукции размещен на сайте пр...,True


True answer: 412
False answer: 409

Test set:


,question,passage
0,Полезна ли ртуть с градусника?,"Отравления ртутью — расстройства здоровья, св..."
1,Являются ли сапрофаги хищниками?,Фауна лесных почв — совокупность видов животны...
2,Водятся ли в индии крокодилы?,"Болотный крокодил, или магер — пресмыкающееся..."
3,Есть ли в батате крахмал?,"Клубневидно вздутые корни весят до 15 кг, сод..."
4,Был ли человек в железной маске?,Остров Сент-Маргерит — крупнейший из Лерински...


### Очистка данных

In [136]:
import unicodedata
class DataCleaner:
    def __init__(self) -> None:
        self.count_removed_symbols = dict()
        self.count_replaced_symbols = dict()
        self.dict_replaced_symbols = dict()
        self.char_to_remove = ['«', '»', '—', ',', '.', '-', '/', ':']
        self.char_to_replace = [['ё', 'е']]

    # функция подсчета количества удаленных символов
    def addReplacedSymbol(self, s_from, s_to = ' '):
        if s_to == ' ':
            if not self.count_removed_symbols.keys().__contains__(s_from):
                self.count_removed_symbols[s_from] = 0
            self.count_removed_symbols[s_from] += 1
        else:
            if not self.count_replaced_symbols.keys().__contains__(s_from):
                self.count_replaced_symbols[s_from] = 0
            self.count_replaced_symbols[s_from] += 1
            self.dict_replaced_symbols[s_from] = s_to

    # удаление знаков ударения и прочих символов unicode
    def unicodeToAscii(self, s):
        tmp = []
        for c in unicodedata.normalize('NFD', s):
            if unicodedata.category(c) != 'Mn':
                tmp.append(c)
            else:
                self.addReplacedSymbol(c)
        return ''.join(tmp)

    # если нужно удалить, то заменяем на пробел чтоб не потерят разделения слов
    def replaceChar(self, s):
        tmp = []
        for i, c in enumerate(s):
            if self.char_to_remove.__contains__(c):
                self.addReplacedSymbol(c, s[i])
                tmp.append(' ')
            else:
                tmp.append(c)
        s = "".join(tmp)

        for s_from, s_to in self.char_to_replace:
            if c == s_from:
                s[i] = s_to
                self.addReplacedSymbol(s_from, s_to)
        return s

    def trimSpaces(self, s):
        while s.__contains__('  '):
            s = s.replace('  ', ' ')
        s = s.strip()
        return s

    def clean(self, df, column):
        for i in range(len(df)):
            df[column][i] = self.unicodeToAscii(df[column][i])
            df[column][i] = df[column][i].lower()
            df[column][i] = self.replaceChar(df[column][i])
            df[column][i] = self.trimSpaces(df[column][i])
        return df

    def summary(self):
        print("===================================")
        print("========= Data Cleaner log ========")
        print("===================================")
        print("===                             ===")
        print("===        Removed Chars        ===")
        print("===                             ===")
        print("===================================")
        
        cols = ["symbol", "count_removed"]
        dfRemoved = pd.DataFrame(columns=cols)
        for c in self.count_removed_symbols:
            current_df = pd.DataFrame([[c, self.count_removed_symbols[c]]], 
                                        columns=cols) 
            dfRemoved = pd.concat([dfRemoved, current_df], ignore_index=True)
        display(dfRemoved)

        print("===================================")
        print("===                             ===")
        print("===        Replaced Chars       ===")
        print("===                             ===")
        print("===================================")
        
        cols = ["symbol_from", "symbol_to", "count_replaced"]
        dfRemoved = pd.DataFrame(columns=cols)
        for c in self.dict_replaced_symbols:
            current_df = pd.DataFrame([[ c, self.dict_replaced_symbols[c], self.count_replaced_symbols[c]]], columns=cols) 
            dfRemoved = pd.concat([dfRemoved, current_df], ignore_index=True)
        display(dfRemoved)

In [172]:
t = DataCleaner()
df_train = t.clean(df_train, 'passage')
df_test = t.clean(df_test, 'passage')
df_validation = t.clean(df_validation, 'passage')
t.summary()

C:\Users\leysh\AppData\Local\Temp\ipykernel_4680\822112328.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = self.unicodeToAscii(df[column][i])
C:\Users\leysh\AppData\Local\Temp\ipykernel_4680\822112328.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = df[column][i].lower()
C:\Users\leysh\AppData\Local\Temp\ipykernel_4680\822112328.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] 

========= Data Cleaner log ========
===                             ===
===        Removed Chars        ===
===                             ===


,symbol,count_removed
0,̆,12576
1,̈,2374
2,́,909
3,̄,4
4,̃,2
5,̀,4
6,̧,1
7,̂,2


===                             ===
===        Replaced Chars       ===
===                             ===


,symbol_from,symbol_to,count_replaced
0,—,—,2447
1,",",",",12296
2,.,.,8857
3,:,:,642
4,-,-,1806
5,«,«,1187
6,»,»,1177
7,/,/,71


In [173]:
df_train['passage'][10]

'а бакингем хоуз и ее коллеги суммировали последствия наиденные в обзорных статьях ранее частые случаи задержки роста плода результатом чего является укороченныи среднии срок беременности и сниженныи вес при рождении по сравнению с нормальными детьми дети 3 4 недельного возраста демонстрируют менее оптимальную двигательную активность рефлексы и ориентацию в пространстве а дети 4 6 лет показывают низкии уровень работы неироповеденческих функции внимания эмоциональнои экспрессии и развития речи и языка величина этих влиянии часто небольшая частично в связи с независимыми переменными включая употребление во время беременности алкоголя табака а также факторы среды у детеи школьного возраста проблемы с устоичивым вниманием и контролем своего поведения а также незначительные с ростом познавательными и языковыми способностями'

In [177]:
df_train

,question,passage,label
0,Вднх - это выставочный центр?,выставочныи центр станция московского монорель...,True
1,Вднх - это выставочный центр?,выставка достижении народного хозяиства в 1959...,True
2,Был ли джиган в black star?,вместе с этим треком они выступили на церемони...,True
3,Xiaomi конкурент apple?,xiaomi китаиская компания основанная в 2010 го...,True
4,Был ли автомат калашникова в вов?,отметив некоторые недостатки и в целом удачную...,False
...,...,...,...
1744,Разрешен ли такой вид ловли акул в настоящее в...,для человека они потенциально полезны в медици...,True
1745,Закреплено ли Гражданство в Конституции,гражданство является одним из институтов конст...,True
1746,"Существуют ли примеры, когда не совсем достато...",в философии под эффективностью понимается спос...,True
1747,Решен ли вопрос о подлинности Диалога Тацита?,в xix веке диалог считали первым произведением...,False


In [179]:
df_train.to_json("DaNetQA/train_c.jsonl", force_ascii=False)
df_test.to_json("DaNetQA/test_c.jsonl", force_ascii=False)
df_validation.to_json("DaNetQA/val_c.jsonl", force_ascii=False)

# Model

In [145]:
import codecs
import json
from sklearn.linear_model import LogisticRegression
import pickle
import joblib

In [ ]:
def save_output(data, path):
    with open(path, mode="w") as file:
        for line in sorted(data, key=lambda x: int(x.get("idx"))):
            line["idx"] = int(line["idx"])
            file.write(f"{json.dumps(line, ensure_ascii=False)}\n")

In [140]:
def build_feature_DaNetQA(row):
    res = str(row["question"]).strip()
    label = row.get("label")
    return res, label

In [141]:
def build_features_DaNetQA(path, vect):
    with codecs.open(path, encoding='utf-8-sig') as reader:
        lines = reader.read().split("\n")
        lines = list(map(json.loads, filter(None, lines)))
    res = list(map(build_feature_DaNetQA, lines))
    texts = list(map(lambda x: x[0], res))
    labels = list(map(lambda x: x[1], res))
    ids = [x["idx"] for x in lines]
    return (vect.transform(texts), labels), ids

In [142]:
def fit_DaNetQA(train, labels):
    clf = LogisticRegression()
    return clf.fit(train, labels)

In [151]:
def eval_DaNetQA(train_path, val_path, test_path, vect):
    train, _ = build_features_DaNetQA(train_path, vect)
    val, _ = build_features_DaNetQA(val_path, vect)
    test, ids = build_features_DaNetQA(test_path, vect)
    clf = fit_DaNetQA(*train)
    try:
        test_score = clf.score(*test)
    except ValueError:
        test_score = None
    test_pred = clf.predict(test[0])
    return clf, {
        "train": clf.score(*train),
        "val": clf.score(*val),
        "test": test_score,
        "test_pred": [{"idx": idx, "label": str(label).lower()} for idx, label in zip(ids, test_pred)]
    }


In [146]:
vect = joblib.load("tfidf.pkl")

c:\Users\leysh\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\leysh\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [147]:
train_path = "DaNetQA/train.jsonl"
val_path = "DaNetQA/val.jsonl"
test_path = "DaNetQA/test.jsonl"

In [168]:
_, DaNetQA_scores = eval_DaNetQA(train_path, val_path, test_path, vect)

In [170]:
DaNetQA_scores

{'train': 0.8010291595197255,
 'val': 0.5907429963459196,
 'test': None,
 'test_pred': [{'idx': 0, 'label': 'true'},
  {'idx': 1, 'label': 'false'},
  {'idx': 2, 'label': 'true'},
  {'idx': 3, 'label': 'true'},
  {'idx': 4, 'label': 'true'},
  {'idx': 5, 'label': 'false'},
  {'idx': 6, 'label': 'true'},
  {'idx': 7, 'label': 'true'},
  {'idx': 8, 'label': 'true'},
  {'idx': 9, 'label': 'true'},
  {'idx': 10, 'label': 'true'},
  {'idx': 11, 'label': 'false'},
  {'idx': 12, 'label': 'true'},
  {'idx': 13, 'label': 'true'},
  {'idx': 14, 'label': 'false'},
  {'idx': 15, 'label': 'false'},
  {'idx': 16, 'label': 'false'},
  {'idx': 17, 'label': 'true'},
  {'idx': 18, 'label': 'true'},
  {'idx': 19, 'label': 'false'},
  {'idx': 20, 'label': 'true'},
  {'idx': 21, 'label': 'true'},
  {'idx': 22, 'label': 'true'},
  {'idx': 23, 'label': 'false'},
  {'idx': 24, 'label': 'false'},
  {'idx': 25, 'label': 'true'},
  {'idx': 26, 'label': 'true'},
  {'idx': 27, 'label': 'true'},
  {'idx': 28, 'labe

In [171]:
DaNetQA_scores["train"], DaNetQA_scores["val"]

0.8010291595197255

In [ ]:
with codecs.open("DaNetQA/train.jsonl", encoding='utf-8-sig') as reader:
    lines = reader.read().split("\n")
    lines = list(map(json.loads, filter(None, lines)))
res = list(map(build_feature_DaNetQA, lines))
texts = list(map(lambda x: x[0], res))
labels = list(map(lambda x: x[1], res))
ids = [x["idx"] for x in lines]
(vect.transform(texts), labels), ids